STEP 1: Build the Heston Model

In [2]:
import numpy as np
import pandas as pd
import sys
import math
import matplotlib as mpl
from scipy.optimize import brute, fmin

1. We define f

In [3]:
i = complex(0, 1)

# To be used in the Heston pricer
def fHeston(s, St, K, r, T, sigma, kappa, theta, volvol, rho):
    # To be used a lot
    prod = rho * sigma * i * s

    # Calculate d
    #d1 = (prod - kappa)**2
    #d2 = (sigma**2) * (i * s + s**2)
    #d = np.sqrt(d1 + d2)
    d = np.sqrt(prod**2-(sigma**2)*(2*i*s-s**2))

    # Calculate g
    g1 = kappa - prod - d
    g2 = kappa - prod + d
    g = g1 / g2

    # Calculate first exponential
    #exp1 = np.exp(np.log(St) * i * s) * np.exp(i * s * r * T)
    exp1 = (St ** (i * s)) * np.exp(i * s * r * T)
    exp2 = 1 - g * np.exp(-d * T)
    exp3 = 1 - g
    mainExp1 = exp1 * np.power(exp2 / exp3, (-2 * theta * kappa) / (sigma**2))

    # Calculate second exponential
    exp4 = theta * kappa * T / (sigma**2)
    exp5 = volvol / (sigma**2)
    exp6 = (1 - np.exp(-d * T)) / (1 - g * np.exp(-d * T))
    mainExp2 = np.exp((exp4 * g1) + (exp5 * g1 * exp6))

    return (mainExp1 * mainExp2)


2. We calculate integral

In [4]:
# Heston Pricer
def priceHestonMid(St, K, r, T, sigma, kappa, theta, volvol, rho):
    P, iterations, maxNumber = 0, 1000, 100
    ds = maxNumber / iterations
    element1 = 0.5 * (St - K * np.exp(-r * T))
    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in range(1, iterations):
        s1 = ds * (2 * j + 1) / 2
        s2 = s1 - i
        #numerator1 = fHeston(s2, St, K, r, T, sigma, kappa, theta, volvol, rho) 
        numerator1 = np.exp(r * T) * fHeston(s2, St, K, r, T, sigma, kappa, theta, volvol, rho)
        numerator2 = K * fHeston(s1, St, K, r, T, sigma, kappa, theta, volvol, rho)
        denominator = np.exp(np.log(K) * i * s1) * i * s1
        P += ds * (numerator1 - numerator2) / denominator
    element2 = P / np.pi
    return np.real((element1 + element2))

STEP 2: Finding Parameters

1. Import XSP data

In [5]:
data = pd.read_csv('/Users/juntao/Downloads/XSP.csv')
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021/5/10,422.829987,423.640015,418.809998,418.839996,418.839996,0.0
1,2021/5/11,NaN,NaN,NaN,NaN,NaN,NaN
2,2021/5/12,413.059998,413.470001,405.690002,406.299988,406.299988,0.0
3,2021/5/13,407.500000,413.160004,407.500000,411.250000,411.250000,0.0
4,2021/5/14,412.959991,418.309998,412.959991,417.390015,417.390015,0.0
5,2021/5/17,416.989990,417.190002,414.269989,416.329987,416.329987,0.0
6,2021/5/18,416.589996,416.920013,412.600006,412.779999,412.779999,0.0
7,2021/5/19,409.850006,411.690002,406.140015,411.570007,411.570007,0.0
8,2021/5/20,NaN,NaN,NaN,NaN,NaN,NaN
9,2021/5/21,416.859985,418.869995,415.170013,415.589996,415.589996,0.0


In [ ]:
sigma, kappa, theta, volvol, rho = 0.1, 0.1, 0.1, 0.1, 0.1